In [1]:
import pandas as pd
import numpy as np

In [2]:
import spacy
from spacy import displacy

In [3]:
fake = pd.read_csv('Fake.csv')
real = pd.read_csv('True.csv')

In [4]:
fake['output'] = 0

In [5]:
fake.head()

,title,text,subject,date,output
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [6]:
real['output'] = 1

In [7]:
real.head()

,title,text,subject,date,output
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
fake.isna().sum()

title      0
text       0
subject    0
date       0
output     0
dtype: int64

In [9]:
real.isna().sum()

title      0
text       0
subject    0
date       0
output     0
dtype: int64

In [10]:
data = pd.concat([fake,real])

In [11]:
x,y = data.drop(columns=['title','subject','date','output']),data['output']

In [12]:
nlp = spacy.load('en_core_web_sm')

In [13]:
from spacy.lang.en.stop_words import STOP_WORDS
import string
stopwords = list(STOP_WORDS)
punct = list(string.punctuation)


def text_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower()
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token+' ')
    output = ''
    return output.join(cleaned_tokens).strip()

In [14]:
x_cleaned = [text_cleaning(i) for i in x['text']]

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [17]:
voc_size = 9000
x_onehot = [one_hot(words,voc_size) for words in x_cleaned]

In [18]:
sent_length = 333
x_embedded = pad_sequences(x_onehot,padding='pre',maxlen=sent_length)

In [19]:
import tensorflow as tf

In [20]:
## Creating model
embedding_vector_features=40
model=tf.keras.Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(tf.keras.layers.LSTM(333))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 333, 40)           360000    
_________________________________________________________________
lstm (LSTM)                  (None, 333)               498168    
_________________________________________________________________
dense (Dense)                (None, 1)                 334       
Total params: 858,502
Trainable params: 858,502
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
x_final=np.array(x_embedded)
y_final=np.array(y)

In [23]:
x_final.shape,y_final.shape

((44898, 333), (44898,))

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.30, random_state=66)

In [26]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
492/492 [==============================] - 27s 51ms/step - loss: 0.3704 - accuracy: 0.8445 - val_loss: 0.0598 - val_accuracy: 0.9829
Epoch 2/10
492/492 [==============================] - 25s 51ms/step - loss: 0.1196 - accuracy: 0.9593 - val_loss: 0.1048 - val_accuracy: 0.9633
Epoch 3/10
492/492 [==============================] - 25s 51ms/step - loss: 0.2398 - accuracy: 0.9195 - val_loss: 0.1243 - val_accuracy: 0.9546
Epoch 4/10
492/492 [==============================] - 25s 51ms/step - loss: 0.0788 - accuracy: 0.9728 - val_loss: 0.1112 - val_accuracy: 0.9576
Epoch 5/10
492/492 [==============================] - 25s 51ms/step - loss: 0.0843 - accuracy: 0.9734 - val_loss: 0.0811 - val_accuracy: 0.9705
Epoch 6/10
492/492 [==============================] - 26s 53ms/step - loss: 0.0201 - accuracy: 0.9942 - val_loss: 0.1850 - val_accuracy: 0.9599
Epoch 7/10
492/492 [==============================] - 25s 51ms/step - loss: 0.0374 - accuracy: 0.9899 - val_loss: 0.0423 - val_accuracy:

In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [28]:
y_pred=model.predict_classes(x_test)

/home/hyper/anaconda3/envs/main/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [29]:
accuracy_score(y_test,y_pred)

0.9911655530809206

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6910
           1       0.99      0.99      0.99      6560

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [34]:
# Creating model
embedding_vector_features=40
model1=tf.keras.Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)))
model1.add(tf.keras.layers.Dropout(0.3))
model1.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 333, 40)           360000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 473,001
Trainable params: 473,001
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
model1.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
492/492 [==============================] - 19s 38ms/step - loss: 0.0623 - accuracy: 0.9777 - val_loss: 0.0708 - val_accuracy: 0.9774
Epoch 2/10
492/492 [==============================] - 19s 38ms/step - loss: 0.0513 - accuracy: 0.9835 - val_loss: 0.0595 - val_accuracy: 0.9833
Epoch 3/10
492/492 [==============================] - 19s 38ms/step - loss: 0.0165 - accuracy: 0.9955 - val_loss: 0.0535 - val_accuracy: 0.9840
Epoch 4/10
492/492 [==============================] - 19s 39ms/step - loss: 0.0098 - accuracy: 0.9975 - val_loss: 0.0572 - val_accuracy: 0.9857
Epoch 5/10
492/492 [==============================] - 19s 39ms/step - loss: 0.0171 - accuracy: 0.9958 - val_loss: 0.1519 - val_accuracy: 0.9510
Epoch 6/10
492/492 [==============================] - 19s 39ms/step - loss: 0.0168 - accuracy: 0.9957 - val_loss: 0.0545 - val_accuracy: 0.9883
Epoch 7/10
492/492 [==============================] - 19s 38ms/step - loss: 0.0170 - accuracy: 0.9956 - val_loss: 0.3188 - val_accuracy:

In [37]:
y_pred1=model1.predict_classes(x_test)

/home/hyper/anaconda3/envs/main/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [38]:
accuracy_score(y_test,y_pred1)

0.9878247958426132

In [39]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6910
           1       0.99      0.99      0.99      6560

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470

